# CS 109a Recommendations

## PROJECT INFO

#### Team Members
Maciej Holubiec, Jimena Romero Pinto, Paul von Chamier

---
#### Problem Statement: 
BLABLA

---

In [46]:
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np
import json
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# LOAD DATA

*** Only for the first time. For later uses skip to the "RELOAD DATA" part. Make sure to download smaller (prepocessed) datasets before running it ***

In [3]:
# LOAD USER
df_user = pd.read_json("user.json", lines = True)

In [ ]:
# LOAD BUSINESS
df_business = pd.read_json("business.json", lines = True)

In [ ]:
# LOAD REVIEWS
with open('review.json', encoding="utf8") as json_file:      
    data_review = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data_review = list(map(json.loads, data_review)) 

df_review = pd.DataFrame(data_review)

# PROCESS

### Select restaurants with more than 30 reviews

In [ ]:
df_business.shape

In [ ]:
df_business = df_business.drop(["hours", "is_open", "latitude", "longitude", "postal_code", "neighborhood", "attributes"], axis = 1)

In [ ]:
df_business_30 = df_business[df_business["review_count"] > 30]

In [ ]:
df_business_30.shape

### Select users who gave more than 100 reviews

In [ ]:
df_users.shape

In [ ]:
df_user = df_user[["user_id", "review_count"]]

In [ ]:
df_users_100 = df_user[df_user["review_count"] > 100]

In [ ]:
df_users_50.shape

### Filter out reviews to those corresponding to selected users and restaurants

In [ ]:
df_review.shape

In [ ]:
df_review = df_review.drop(["cool", "date", "funny", "review_id", "text", "useful"], axis = 1)

In [ ]:
df_review.head()

In [ ]:
df_review_30_100 = df_review[df_review["user_id"].isin(df_users_50["user_id"])]

In [ ]:
df_review_30_100 = df_review_30_100[df_review_30_100["user_id"].isin(df_business_30["business_id"])]

In [ ]:
df_review.shape

### Save data for future reference so we deal with smaller files

In [ ]:
df_users_100.to_json("df_user_100.json")

In [ ]:
df_business_30.to_json("df_business_30.json")

In [ ]:
df_review_30_100.to_json("df_review_30_100.json")

# RELOAD DATA

In [4]:
df_user = pd.read_json("df_user_100.json")

In [5]:
df_business = pd.read_json("df_business_30.json")

In [6]:
df_review = pd.read_json("df_review_30_100.json")

### Sample from the data frame because the dataset is still too big

In [7]:
fraction_of_df = 0.01

In [8]:
np.random.seed(9001)
df_review_small = df_review.sample(frac=fraction_of_df)

In [26]:
df_review_small.shape

(9241, 3)

### Create latent matrix

In [10]:
r_df = df_review_small.pivot(index = 'user_id', columns ='business_id', values = 'stars')
r_df.head()

business_id,--9e1ONYQuAa-CB_Rrw7Tw,--I7YYLada0tSLkORTHb5Q,-050d_XIor1NpCuWkbIVaQ,-0BxAGlIk5DJAGVkpqBXxg,-0WegMt6Cy966qlDKhu6jA,-1UMR00eXtwaeh59pEiDjA,-3zffZUHoY8bQjGfPSoBKQ,-4TMQnQJW1yd6NqGRDvAeA,-4gwgdvSq_PqOmHb70au8Q,-6h3K1hj0d4DRcZNUtHDuw,...,zvO-PJCpNk4fgAVUnExYAA,zvQIEpJUmLLmMMffNntHXQ,zvxaG-BIuxo5Hm6UO2yG8Q,zw9_mqWBn1QCfZg88w0Exg,zwNC-Ow4eIMan2__bS9-rg,zxsktAMsVBKj2PvvCV11UQ,zz3CqZhNx2rQ_Yp6zHze-A,zzSYBWuv_fXGtSgsO-6_1g,zzf3RkMI1Y2E1QaZqeU8yA,zzzaIBwimxVej4tY6qFOUQ
user_id,,,,,,,,,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--2vR0DIsmQ6WfcSzKWigw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--Qh8yKWAvIP4V4K8ZPfHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ZNfWKj1VyVElRx6-g1fg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-21ooEUDqBkCF0hr94Pvgg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
r_df.shape

(6472, 6064)

# MODELS

### Baseline Averages

In [42]:
avg_mean = r_df.mean().mean()
avg_mean

3.7397189207325456

In [43]:
rows_length = r_df.shape[0]
cols_length = r_df.shape[1]

In [46]:
cols_means = r_df.mean(axis = 0)
rows_means = r_df.mean(axis = 1)

In [62]:
cols_means.head()

business_id
--9e1ONYQuAa-CB_Rrw7Tw    3.500000
--I7YYLada0tSLkORTHb5Q    4.000000
-050d_XIor1NpCuWkbIVaQ    3.666667
-0BxAGlIk5DJAGVkpqBXxg    3.000000
-0WegMt6Cy966qlDKhu6jA    2.000000
dtype: float64

In [63]:
rows_means.head()

user_id
---1lKK3aKOuomHnwAkAow    4.0
--2vR0DIsmQ6WfcSzKWigw    3.5
--Qh8yKWAvIP4V4K8ZPfHA    3.0
--ZNfWKj1VyVElRx6-g1fg    5.0
-21ooEUDqBkCF0hr94Pvgg    5.0
dtype: float64

In [65]:
preds_array_avg = np.fromfunction(lambda i, j: rows_means[i] + cols_means[j] - avg_mean, (rows_length, cols_length), dtype=int)

In [68]:
preds_array_avg

array([[ 3.76028108,  4.26028108,  3.92694775, ...,  5.26028108,
         4.26028108,  4.26028108],
       [ 3.26028108,  3.76028108,  3.42694775, ...,  4.76028108,
         3.76028108,  3.76028108],
       [ 2.76028108,  3.26028108,  2.92694775, ...,  4.26028108,
         3.26028108,  3.26028108],
       ..., 
       [ 1.76028108,  2.26028108,  1.92694775, ...,  3.26028108,
         2.26028108,  2.26028108],
       [ 2.76028108,  3.26028108,  2.92694775, ...,  4.26028108,
         3.26028108,  3.26028108],
       [ 3.76028108,  4.26028108,  3.92694775, ...,  5.26028108,
         4.26028108,  4.26028108]])

In [54]:
# fill_na_array = np.zeros((rows_length,cols_length))
# for i in range(0,rows_length):
#     for j in range(0,cols_length):
#         fill_na_array[i][j] = rows_means[i] + cols_means[j] - avg_mean
#     if (i%1000 == 0):
#         print(i)

In [58]:
avg_preds_df = pd.DataFrame(preds_array_avg, columns = r_df.columns, index = r_df.index)

In [59]:
avg_preds_df.head()

business_id,--9e1ONYQuAa-CB_Rrw7Tw,--I7YYLada0tSLkORTHb5Q,-050d_XIor1NpCuWkbIVaQ,-0BxAGlIk5DJAGVkpqBXxg,-0WegMt6Cy966qlDKhu6jA,-1UMR00eXtwaeh59pEiDjA,-3zffZUHoY8bQjGfPSoBKQ,-4TMQnQJW1yd6NqGRDvAeA,-4gwgdvSq_PqOmHb70au8Q,-6h3K1hj0d4DRcZNUtHDuw,...,zvO-PJCpNk4fgAVUnExYAA,zvQIEpJUmLLmMMffNntHXQ,zvxaG-BIuxo5Hm6UO2yG8Q,zw9_mqWBn1QCfZg88w0Exg,zwNC-Ow4eIMan2__bS9-rg,zxsktAMsVBKj2PvvCV11UQ,zz3CqZhNx2rQ_Yp6zHze-A,zzSYBWuv_fXGtSgsO-6_1g,zzf3RkMI1Y2E1QaZqeU8yA,zzzaIBwimxVej4tY6qFOUQ
user_id,,,,,,,,,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,3.760281,4.260281,3.926948,3.260281,2.260281,4.260281,4.260281,4.260281,2.260281,3.260281,...,5.260281,3.260281,5.260281,3.260281,3.260281,4.260281,4.260281,5.260281,4.260281,4.260281
--2vR0DIsmQ6WfcSzKWigw,3.260281,3.760281,3.426948,2.760281,1.760281,3.760281,3.760281,3.760281,1.760281,2.760281,...,4.760281,2.760281,4.760281,2.760281,2.760281,3.760281,3.760281,4.760281,3.760281,3.760281
--Qh8yKWAvIP4V4K8ZPfHA,2.760281,3.260281,2.926948,2.260281,1.260281,3.260281,3.260281,3.260281,1.260281,2.260281,...,4.260281,2.260281,4.260281,2.260281,2.260281,3.260281,3.260281,4.260281,3.260281,3.260281
--ZNfWKj1VyVElRx6-g1fg,4.760281,5.260281,4.926948,4.260281,3.260281,5.260281,5.260281,5.260281,3.260281,4.260281,...,6.260281,4.260281,6.260281,4.260281,4.260281,5.260281,5.260281,6.260281,5.260281,5.260281
-21ooEUDqBkCF0hr94Pvgg,4.760281,5.260281,4.926948,4.260281,3.260281,5.260281,5.260281,5.260281,3.260281,4.260281,...,6.260281,4.260281,6.260281,4.260281,4.260281,5.260281,5.260281,6.260281,5.260281,5.260281


### Baseline Regression

In [28]:
categorical_columns = ['business_id', 'user_id']
df_review = pd.get_dummies(df_review_small, columns=categorical_columns, drop_first=False)

In [31]:
df_review.shape

(9241, 12537)

In [32]:
df_review.head()

,stars,business_id_--9e1ONYQuAa-CB_Rrw7Tw,business_id_--I7YYLada0tSLkORTHb5Q,business_id_-050d_XIor1NpCuWkbIVaQ,business_id_-0BxAGlIk5DJAGVkpqBXxg,business_id_-0WegMt6Cy966qlDKhu6jA,business_id_-1UMR00eXtwaeh59pEiDjA,business_id_-3zffZUHoY8bQjGfPSoBKQ,business_id_-4TMQnQJW1yd6NqGRDvAeA,business_id_-4gwgdvSq_PqOmHb70au8Q,...,user_id_zwhty_ZmxbHAHoDfMjNbag,user_id_zwpP3Lt8USKS91Ds52G8UA,user_id_zx3y74_pvIRuQSVIrgzCew,user_id_zxmIKyEX89aY0QLC1JJksA,user_id_zxuKNsouUK2zCf1GweuVNw,user_id_zyFN0KU7Y9MfTZIROrsKhA,user_id_zyJWc8oW8JpPg2xXMQsnAA,user_id_zzPvEodjvLPe-5NvV0U_cg,user_id_zzaq5Fn1U2Feut3dGxidNg,user_id_zzpgpo54-_P-4rzzBtOuLQ
1264734,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1929700,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1938284,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301192,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2977242,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
#Creating train and test

np.random.seed(9001)
msk = np.random.rand(len(df_review)) < 0.5

# data_train = df_subset[msk]
# data_test = df_subset[~msk]

x_train = df_review[msk].drop(['stars'], axis=1) # DataFrame

x_test = df_review[~msk].drop(['stars'], axis=1) # DataFrame

y_train = df_review[msk].stars #series

y_test = df_review[~msk].stars # series

In [34]:
x_train.head()

,business_id_--9e1ONYQuAa-CB_Rrw7Tw,business_id_--I7YYLada0tSLkORTHb5Q,business_id_-050d_XIor1NpCuWkbIVaQ,business_id_-0BxAGlIk5DJAGVkpqBXxg,business_id_-0WegMt6Cy966qlDKhu6jA,business_id_-1UMR00eXtwaeh59pEiDjA,business_id_-3zffZUHoY8bQjGfPSoBKQ,business_id_-4TMQnQJW1yd6NqGRDvAeA,business_id_-4gwgdvSq_PqOmHb70au8Q,business_id_-6h3K1hj0d4DRcZNUtHDuw,...,user_id_zwhty_ZmxbHAHoDfMjNbag,user_id_zwpP3Lt8USKS91Ds52G8UA,user_id_zx3y74_pvIRuQSVIrgzCew,user_id_zxmIKyEX89aY0QLC1JJksA,user_id_zxuKNsouUK2zCf1GweuVNw,user_id_zyFN0KU7Y9MfTZIROrsKhA,user_id_zyJWc8oW8JpPg2xXMQsnAA,user_id_zzPvEodjvLPe-5NvV0U_cg,user_id_zzaq5Fn1U2Feut3dGxidNg,user_id_zzpgpo54-_P-4rzzBtOuLQ
1264734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3499589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3189998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3946843,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2861654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
def R_square(model, x, y):
    predict = model.predict(x)
    r2 = r2_score(y, predict)
    return r2

In [39]:
#ols lasso
resultados_ols_l = pd.DataFrame()


from sklearn.model_selection import KFold
cv=KFold(n_splits = 5)
for train_obs, valid_obs in cv.split(x_train, y_train):
    x_train_l = x_train.iloc[train_obs,:]
    x_valid = x_train.iloc[valid_obs,:]
    y_train_l = y_train.iloc[train_obs]
    y_valid = y_train.iloc[valid_obs]
    
    for alpha in [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]:
        ols_lasso = Lasso(alpha=alpha)
        ols_lasso.fit(x_train_l,y_train_l)
        resultados_for_loop = pd.DataFrame(dict(model =["ols lasso"],
                                                parameter=[alpha], 
                                                R2=[R_square(ols_lasso, x_valid, y_valid)]))
        resultados_ols_l = resultados_ols_l.append(resultados_for_loop)
        
resultados_ols_l = resultados_ols_l.groupby(by=["parameter","model"]).mean()
resultados_ols_l


,,R2
parameter,model,
0.00001,ols lasso,-0.186302
0.00010,ols lasso,-0.084875
0.00100,ols lasso,-0.003588
0.01000,ols lasso,-0.000160
0.10000,ols lasso,-0.000160
1.00000,ols lasso,-0.000160
10.00000,ols lasso,-0.000160
100.00000,ols lasso,-0.000160
1000.00000,ols lasso,-0.000160


In [47]:
def mse(model, x, y):
    predict = model.predict(x)
    mse = mean_squared_error(y, predict)
    return mse

In [ ]:
resultados_ols_2 = pd.DataFrame()

cv=KFold(n_splits = 5)
for train_obs, valid_obs in cv.split(x_train, y_train):
    x_train_l = x_train.iloc[train_obs,:]
    x_valid = x_train.iloc[valid_obs,:]
    y_train_l = y_train.iloc[train_obs]
    y_valid = y_train.iloc[valid_obs]
    
    for alpha in [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]:
        ols_lasso = Lasso(alpha=alpha)
        ols_lasso.fit(x_train_l,y_train_l)
        resultados_for_loop = pd.DataFrame(dict(model =["ols lasso"],
                                                parameter=[alpha], 
                                                MSE=[mse(ols_lasso, x_valid, y_valid)]))
        resultados_ols_2 = resultados_ols_2.append(resultados_for_loop)
        
resultados_ols_2 = resultados_ols_2.groupby(by=["parameter","model"]).mean()
resultados_ols_2

C:\Users\jimen\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [40]:
params = {'alpha': [0.01, 0.1, 1, 10, 100]}
model_lasso = Lasso()
clf1 = GridSearchCV(model_lasso, param_grid = params, scoring = 'mean_squared_error')
clf1.fit(x_train, y_train)

C:\Users\jimen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\jimen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\jimen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\jimen\Anaconda3\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\jimen\Anaconda3\lib\site-pa

GridSearchCV(cv=None, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='mean_squared_error', verbose=0)

### Matrix Factorization

In [97]:
fill_zero_rf = r_df.fillna(0)

In [100]:
Q = fill_zero_rf.values

In [103]:
W = Q>0.5
W[W == True] = 1
W[W == False] = 0
# To be consistent with our Q matrix
W = W.astype(np.float64, copy=False)

In [105]:
W.shape

(6472, 6064)

In [106]:
lambda_ = 0.1
n_factors = 100
m, n = Q.shape
n_iterations = 20

In [107]:
X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)

In [109]:
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)

In [110]:
errors = []
for ii in range(n_iterations):
    X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
                        np.dot(Y, Q.T)).T
    Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
                        np.dot(X.T, Q))
    if ii % 100 == 0:
        print('{}th iteration is completed'.format(ii))
    errors.append(get_error(Q, X, Y, W))
Q_hat = np.dot(X, Y)
print('Error of rated movies: {}'.format(get_error(Q, X, Y, W)))

0th iteration is completed
Error of rated movies: 119925.73796658634


In [113]:
fac_preds_df = pd.DataFrame(Q_hat, columns = r_df.columns, index = r_df.index)

In [115]:
fac_preds_df.head()

business_id,--9e1ONYQuAa-CB_Rrw7Tw,--I7YYLada0tSLkORTHb5Q,-050d_XIor1NpCuWkbIVaQ,-0BxAGlIk5DJAGVkpqBXxg,-0WegMt6Cy966qlDKhu6jA,-1UMR00eXtwaeh59pEiDjA,-3zffZUHoY8bQjGfPSoBKQ,-4TMQnQJW1yd6NqGRDvAeA,-4gwgdvSq_PqOmHb70au8Q,-6h3K1hj0d4DRcZNUtHDuw,...,zvO-PJCpNk4fgAVUnExYAA,zvQIEpJUmLLmMMffNntHXQ,zvxaG-BIuxo5Hm6UO2yG8Q,zw9_mqWBn1QCfZg88w0Exg,zwNC-Ow4eIMan2__bS9-rg,zxsktAMsVBKj2PvvCV11UQ,zz3CqZhNx2rQ_Yp6zHze-A,zzSYBWuv_fXGtSgsO-6_1g,zzf3RkMI1Y2E1QaZqeU8yA,zzzaIBwimxVej4tY6qFOUQ
user_id,,,,,,,,,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,7.943997e-04,-1.518564e-21,1.969040e-06,1.908522e-21,-3.990023e-22,3.932058e-21,1.589589e-07,-4.588570e-21,-1.412380e-34,-1.415326e-05,...,3.269211e-17,-3.889743e-06,4.401867e-04,9.758885e-04,2.006404e-08,2.916327e-19,1.987112e-11,-1.893097e-17,-9.051990e-22,4.487250e-21
--2vR0DIsmQ6WfcSzKWigw,1.086836e-03,-1.211306e-21,1.007358e-05,-1.510127e-20,-1.712800e-22,5.011868e-21,1.193260e-05,4.054704e-21,-2.478428e-33,9.449312e-06,...,-1.026121e-16,1.082404e-03,-4.154001e-04,-4.782610e-04,7.683776e-07,8.325802e-19,2.565381e-11,5.646294e-17,7.744049e-21,4.190921e-21
--Qh8yKWAvIP4V4K8ZPfHA,-3.107882e-05,-5.561645e-22,-5.770952e-07,2.341000e-21,-1.577538e-23,3.172741e-22,-1.229813e-07,7.129464e-23,-1.028670e-34,6.775438e-07,...,9.067241e-18,-6.001753e-07,-5.185921e-05,-1.296155e-04,-1.047686e-09,3.360838e-20,2.202390e-12,-5.552175e-18,6.888308e-22,9.146683e-22
--ZNfWKj1VyVElRx6-g1fg,-1.514534e-04,-2.947673e-21,1.094528e-05,6.712238e-21,6.435715e-22,-7.392537e-22,-6.250106e-06,-3.095135e-21,1.099674e-33,-4.556872e-06,...,7.176194e-17,-9.171175e-05,1.430555e-03,-3.989327e-03,3.027983e-07,-5.807685e-19,1.675468e-11,-8.881134e-18,-7.069700e-21,-2.454638e-22
-21ooEUDqBkCF0hr94Pvgg,1.152899e-15,-8.311287e-33,-4.832679e-18,1.978269e-32,-6.009178e-34,3.225216e-33,-1.057801e-18,-9.867903e-33,-3.994300e-46,-1.019357e-17,...,2.435287e-29,-2.960533e-17,1.978350e-15,-1.594140e-17,-5.296077e-20,4.891656e-31,7.776171e-24,2.160129e-29,-5.469132e-33,6.118949e-33


# PREDICT

In [93]:
def get_reccomendations(user, number_rec,df):
    top_preds = df.loc[user][fill_zero_rf.loc[user] == 0].sort_values(ascending = False)[:number_rec]
    top_preds_df = pd.DataFrame(top_preds).rename(columns={user:"predicted rating"})
    predictions = pd.merge(left = top_preds_df, right = df_business, left_index = True, right_on="business_id")
#     top_preds_df.join(df_business, on="business_id")
    return predictions
#     return top_preds_df

In [94]:
get_reccomendations("---1lKK3aKOuomHnwAkAow",10,avg_preds_df)

,predicted rating,address,business_id,categories,city,name,review_count,stars,state
105078,5.260281,109 N Walnut St,e0prCZXtHGQIKeQ_wTW3uw,"[Food, Coffee & Tea]",Champaign,Cafe Kopi,152,4.0,IL
2924,5.260281,11444 Uptown Ave,oVZsUPIMdS8Vk7Jip5l0_g,"[Ice Cream & Frozen Yogurt, Food]",Cleveland,Mitchell's Homemade Ice Cream,79,4.5,OH
133340,5.260281,812 Queen Street E,SCklYa_jhihWAcfIn-_chg,"[French, Breakfast & Brunch, Restaurants]",Toronto,Bonjour Brioche,258,3.5,ON
120090,5.260281,617 E Green St,SCxtzXcvyk1hygdmOZqd1A,"[Restaurants, Mediterranean, Arabian]",Champaign,Mashawi Grill,90,4.5,IL
148584,5.260281,85 W Boston St,CZKHXlDuy3IagC2W881fyA,"[Cocktail Bars, Bars, American (New), Nightlif...",Chandler,The Brickyard Downtown,258,4.5,AZ
38842,5.260281,3401 Dufferin Street,CVhTzLShRr_LDR5vjFXdcA,"[Mobile Phones, Local Services, Electronics, S...",North York,Apple Store,41,2.0,ON
134572,5.260281,3545 Las Vegas Blvd S,CVKOPzBVOj3_apFUmZ9ZWw,"[Fish & Chips, Comfort Food, Seafood, Restaura...",Las Vegas,Gordon Ramsay Fish & Chips,454,4.0,NV
5210,5.260281,1500 N Green Valley Pkwy,oAngC9V69vyzBVYwKEJQaw,"[Chinese, Restaurants]",Henderson,China Tango,187,4.0,NV
61182,5.260281,"2763 East Carson St, Unit C-105",oCvxJBOs-Mbg-4oanPO7lQ,"[Greek, Restaurants, Fast Food, Sandwiches, Me...",Pittsburgh,Pita Pit,39,4.0,PA
98832,5.260281,"4727 E Bell Rd, Ste 45308",CQxhyLM833WF45yrS9yeAA,"[Pets, Veterinarians, Pet Services, Pet Hospice]",Phoenix,Happy Endings In-Home Pet Euthanasia,74,5.0,AZ


In [116]:
get_reccomendations("---1lKK3aKOuomHnwAkAow",10,fac_preds_df)

,predicted rating,address,business_id,categories,city,name,review_count,stars,state
92918,0.018586,"953 E Sahara Ave, Ste A5",KskYqH1Bi7Z_61pH6Om8pg,"[Automotive, Car Dealers, Restaurants, Thai, N...",Las Vegas,Lotus of Siam,3838,4.0,NV
18665,0.010622,1197 W Carson St,PU-CSnMYXizOS3uhr316eg,"[Tours, Public Transportation, Landmarks & His...",Pittsburgh,Duquesne Incline,274,4.5,PA
139666,0.010555,2411 W Sahara Ave,awI4hHMfa7H0Xf0-ChU5hg,"[Restaurants, Nightlife, Seafood, Bars]",Las Vegas,Oyster Bar,1974,4.5,NV
139596,0.007590,3655 Las Vegas Blvd S,ii8sAGBexBOJoYRFafF9XQ,"[Event Planning & Services, Casinos, Hotels & ...",Las Vegas,Paris Las Vegas Hotel & Casino,2013,3.0,NV
69590,0.006220,3525 Liberty Ave,ejaUQ1hYo7Q7xCL1HdPINw,"[American (Traditional), Food, Restaurants, De...",Pittsburgh,Church Brew Works,858,3.5,PA
67633,0.005507,7435 S Durango Dr,16tWXenJ1wTqugXJU7yiZg,"[Restaurants, Sushi Bars, Desserts, Food]",Las Vegas,808 Sushi,435,3.5,NV
70249,0.005376,1770 Festival Plaza Dr,tMYYSWyTppDcF070Hmy-kQ,"[Restaurants, American (New), Chicken Shop, Ch...",Las Vegas,Blue Ribbon Fried Chicken,334,3.5,NV
114966,0.005167,"1716 W Horizon Ridge Pkwy, Ste 100",uBdYMY6a6A7FyxzTSwOiDg,"[Breakfast & Brunch, Restaurants, American (New)]",Henderson,Kitchen Table,687,4.0,NV
8885,0.005164,3545 Las Vegas Blvd S,Ki8seQY5mtXc36xjoMfeDA,"[Gastropubs, Restaurants, Italian, Pizza]",Las Vegas,Flour & Barley,408,3.5,NV
122843,0.005002,7400 Las Vegas Blvd S,9edPSkfXKsJmkZYIaOmA7Q,"[Restaurants, Outlet Stores, Fashion, Shopping...",Las Vegas,Las Vegas South Premium Outlets,364,3.5,NV
